In [1]:
##### DNN module

from keras.utils import to_categorical
from keras import backend as K
from keras.models import Model
from keras.applications import ResNet50
from keras.optimizers import Adagrad, RMSprop, Adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import time
from PIL import Image
from scipy.interpolate import RegularGridInterpolator
import pandas as pd

In [4]:
batch_size = 20
epochs = 30
num_class = 5

In [5]:
pic_size = 200
fn_serial = '012'

In [6]:
net = ResNet50(include_top=False,
               weights='imagenet',
               #input_tensor=None,
               input_shape=(pic_size ,pic_size, 3))

C:\Users\cpjohn\.conda\envs\cpjohn\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [7]:
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(num_class, activation='softmax', name='softmax')(x)

In [8]:
net_final = Model(inputs=net.input, outputs=output_layer)

In [9]:
net_final.compile(optimizer=RMSprop(), # Adam(lr=1e-5)
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [10]:
print(net_final.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [11]:
augment_generator = ImageDataGenerator(
                                       rotation_range=20,
                                       rescale=1./255,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip=True,
                                       featurewise_center=True,
                                       featurewise_std_normalization=True,
                                       data_format='channels_last',
                                      )

In [12]:
train_generator = augment_generator.flow_from_directory('database/image_data/train',
                                                        target_size=(pic_size, pic_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        )

Found 2823 images belonging to 5 classes.


In [13]:
valid_generator = augment_generator.flow_from_directory('database/image_data/train',
                                                        target_size=(pic_size, pic_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        )

Found 2823 images belonging to 5 classes.


In [14]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

C:\Users\cpjohn\.conda\envs\cpjohn\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\cpjohn\.conda\envs\cpjohn\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


data batch shape: (20, 200, 200, 3)
labels batch shape: (20, 5)


In [15]:
history = net_final.fit(train_generator,
                        steps_per_epoch=int(2823/batch_size),
                        epochs=epochs,
                        verbose=1,
                        validation_data=valid_generator,
                        validation_steps=50
                       )

Epoch 1/30


C:\Users\cpjohn\.conda\envs\cpjohn\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\cpjohn\.conda\envs\cpjohn\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


141/141 [==============================] - 54s 382ms/step - loss: 5.7147 - accuracy: 0.3564 - val_loss: 359657.1875 - val_accuracy: 0.1730
Epoch 2/30
141/141 [==============================] - 37s 265ms/step - loss: 4.9175 - accuracy: 0.3582 - val_loss: 2163.4878 - val_accuracy: 0.1680
Epoch 3/30
141/141 [==============================] - 37s 264ms/step - loss: 4.3655 - accuracy: 0.4006 - val_loss: 1.6148 - val_accuracy: 0.2503
Epoch 4/30
141/141 [==============================] - 38s 267ms/step - loss: 4.8210 - accuracy: 0.3796 - val_loss: 1.5891 - val_accuracy: 0.2440
Epoch 5/30
141/141 [==============================] - 37s 265ms/step - loss: 4.1610 - accuracy: 0.4010 - val_loss: 1.5823 - val_accuracy: 0.2460
Epoch 6/30
141/141 [==============================] - 37s 264ms/step - loss: 3.2546 - accuracy: 0.4303 - val_loss: 1.5477 - val_accuracy: 0.3510
Epoch 7/30
141/141 [==============================] - 37s 265ms/step - loss: 3.2042 - accuracy: 0.4481 - val_loss: 2.9115 - val_accur

In [16]:
'''
history =  net_final.fit_generator(train_generator,
                                   steps_per_epoch=int(2823/batch_size),
                                   epochs=epochs,
                                   verbose=1,
                                   #validation_data=valid_generator,
                                   #validation_steps=50
                                  )
'''

'\nhistory =  net_final.fit_generator(train_generator,\n                                   steps_per_epoch=int(2823/batch_size),\n                                   epochs=epochs,\n                                   verbose=1,\n                                   #validation_data=valid_generator,\n                                   #validation_steps=50\n                                  )\n'

In [17]:
#model.save(f'model/STD_datagen_1230_{fn_serial}.h5')

In [18]:
##### Test on unknown sample

In [19]:
path_unknown = glob.glob('database/image_data/test/*')
path_unknown[:5]

['database/image_data/test\\0028624c49b3e0610ff9f1d111f5d532.jpg',
 'database/image_data/test\\002c30700185b7971369258b438070d5.jpg',
 'database/image_data/test\\00852f4f666acecd0c0d140365b42efd.jpg',
 'database/image_data/test\\00c08828fce04e360c732cac01edad9e.jpg',
 'database/image_data/test\\00d366e7877b6a78b104b57d67b60e6b.jpg']

In [20]:
def resampleRGI3d(input_mx, resize_to, dtype='float32'):
    # Ref: 10 3-A-7
    # Ver. 2.1
    # input_mx : numpy array, the original target matrix
    # resize_to: list or tuple with 3 int inside
    a, b, c = np.shape(input_mx)
    p, q, r = resize_to
    z_grid = np.linspace(0, p - 1, a)
    y_grid = np.linspace(0, q - 1, b)
    x_grid = np.linspace(0, r - 1, c)
    RGI = RegularGridInterpolator((z_grid, y_grid, x_grid), input_mx)
    z_grid_t2 = np.arange(p)
    y_grid_t2 = np.arange(q)
    x_grid_t2 = np.arange(r)
    meshgrid_para = np.meshgrid(z_grid_t2, y_grid_t2, x_grid_t2)
    RGI_mesh_mx = RGI((meshgrid_para[0], meshgrid_para[1], meshgrid_para[2]))
    RGI_mx = np.transpose(RGI_mesh_mx, axes=[1, 0, 2]).astype(dtype)
    return RGI_mx

def happy_time(start,stop):
    process_time = round(stop - start)
    ss = process_time % 60
    mm = process_time // 60 % 60
    hh = process_time // 3600
    duration = "Process time == {}s == {}H {}m {}s".format(process_time,hh,mm,ss)
    return duration

In [21]:

id_li = []
flower_class = []
loop_start = time.time()

for j in range(len(path_unknown)):
    serial = os.path.basename(path_unknown[j])[:-4]
    temp_pic = np.asarray(Image.open(path_unknown[j]))
    temp_pic = resampleRGI3d(temp_pic, (pic_size, pic_size, 3))
    temp_shape = np.shape(temp_pic)
    temp_pic = np.expand_dims(temp_pic, axis = 0)
    pred = net_final.predict(temp_pic)[0]
    id_li.append(serial)
    flower_class.append(list(pred).index(max(pred)))
    if (j+1) % 200 == 0:
        print(j+1, 'files done.', happy_time(loop_start, time.time()))

200 files done. Process time == 13s == 0H 0m 13s
400 files done. Process time == 25s == 0H 0m 25s
600 files done. Process time == 36s == 0H 0m 36s
800 files done. Process time == 47s == 0H 0m 47s
1000 files done. Process time == 58s == 0H 0m 58s
1200 files done. Process time == 69s == 0H 1m 9s
1400 files done. Process time == 80s == 0H 1m 20s
1600 files done. Process time == 92s == 0H 1m 32s
1800 files done. Process time == 103s == 0H 1m 43s
2000 files done. Process time == 114s == 0H 1m 54s


In [22]:
pred_result_df = pd.DataFrame(columns=['id', 'flower_class'])
pred_result_df['id'] = id_li
pred_result_df['flower_class'] = flower_class
pred_result_df

,id,flower_class
0,0028624c49b3e0610ff9f1d111f5d532,3
1,002c30700185b7971369258b438070d5,3
2,00852f4f666acecd0c0d140365b42efd,3
3,00c08828fce04e360c732cac01edad9e,3
4,00d366e7877b6a78b104b57d67b60e6b,3
...,...,...
1995,ff7eac29b6d7a33fbd8009677c3e9c58,3
1996,ffbc32a7b67dfe72b8d35d4b1b35fd6c,3
1997,ffea1f275c05accb0a6bfd1203620c7e,3
1998,ffeb2a1cf53464b6af937ab8af0c2946,3


In [23]:
pred_result_df['flower_class'].value_counts().sort_index()

#0    574
#1      3
#2    196
#3    687
#4    540
#Name: flower_class, dtype: int64

3    2000
Name: flower_class, dtype: int64

In [24]:
pred_result_df.to_csv(f'model/pred_result_STD_datagen_1230_{fn_serial}.csv', index=False)

In [25]:
#####
#####
#####